# Task 2: Understand body language by gesture recognition with convolutional neural network

## 1. Do literature search on Convolution Neural Network. Learn how to build a convolutional layer in PyTorch.

## 2. Referring to the guide in Task 1, build your own network for gesture classification using convolutional layers. Please see the references 4 in the manual to learn how to build convolutional layers in PyTorch.

## 3. Analyse and comment on the performance of the model. Make a comparison between the fully connected based and convolutional based models and comment on it.

In [2]:
import cv2
import numpy as np
import os
import itertools
import torch.utils.data as utils_data

# 1) data preprocessing

In [3]:
def processSkinImage(filePath, resize_HW=48):
    # step 1
    # read the image
    original = cv2.imread(filename=filePath)

    # step 2
    # resize the image to
    image_resized = cv2.resize(original, (resize_HW, resize_HW))

    # step 3
    # convert the image from rgb to YCbCr
    image_ycbcr = cv2.cvtColor(image_resized, cv2.COLOR_BGR2YCR_CB)

    # step 4
    # get the central color of the image
    # expected the hand to be in the central of the image
    Cb_center_color = image_ycbcr[int(resize_HW/2), int(resize_HW/2), 1]
    Cr_center_color = image_ycbcr[int(resize_HW/2), int(resize_HW/2), 2]
    # set the range
    Cb_Difference = 15
    Cr_Difference = 10

    # step 5
    # detect skin pixels
    Cb = image_ycbcr[:, :, 1]
    Cr = image_ycbcr[:, :, 2]
    index = np.where((Cb >= Cb_center_color-Cb_Difference) & (Cb <= Cb_center_color+Cb_Difference)
                     & (Cr >= Cr_center_color-Cr_Difference) & (Cr <= Cr_center_color+Cr_Difference))

    # Mark detected pixels and output
    image_output = np.zeros((resize_HW, resize_HW))
    image_output[index] = 255

    # show image
    # cv2.imshow("", image_output)
    # cv2.waitKey(0)
    
    return image_output

In [4]:
path = './dataset/images'
path_processed = './dataset_processed/images'

# -------------------images processing--------------
for mainDir, subDir, fileList in os.walk(path):
    for file in fileList:
        currentPath = os.path.join(mainDir, file)
        print(currentPath)
        processedImage = processSkinImage(currentPath)

        new_mainDir = path_processed + mainDir.split(path)[-1]
        if not os.path.exists(new_mainDir):
            os.makedirs(new_mainDir)
        cv2.imwrite(os.path.join(new_mainDir, file), processedImage)

# -----------------label generation----------------
label_path = './dataset_processed/labels'
if not os.path.exists(label_path):
    os.makedirs(label_path)

files = os.listdir(path)
for i, file in enumerate(files):
    if file == '.DS_Store':
        continue
    subclass_label_path = os.path.join(label_path, file+'.txt')
    with open(subclass_label_path, 'w') as f:
        f.write('#label\n')
    for _ in range(len(os.listdir(os.path.join(path_processed, file)))):
        with open(subclass_label_path, 'a') as f:
            f.write('{:d}\n'.format(i))
    f.close()

./dataset/images/right/right (4).jpg
./dataset/images/right/right (13).jpg
./dataset/images/right/right (8).jpg
./dataset/images/right/right (24).jpg
./dataset/images/right/right (9).jpg
./dataset/images/right/right (12).jpg
./dataset/images/right/right (5).jpg
./dataset/images/right/right (23).jpg
./dataset/images/right/right (19).jpg
./dataset/images/right/right (2).jpg
./dataset/images/right/right (15).jpg
./dataset/images/right/right (14).jpg
./dataset/images/right/right (3).jpg
./dataset/images/right/right (18).jpg
./dataset/images/right/right (22).jpg
./dataset/images/right/right (21).jpg
./dataset/images/right/right (17).jpg
./dataset/images/right/right (1).jpg
./dataset/images/right/right (16).jpg
./dataset/images/right/right (20).jpg
./dataset/images/right/right (11).jpg
./dataset/images/right/right (6).jpg
./dataset/images/right/right (7).jpg
./dataset/images/right/right (10).jpg
./dataset/images/left/left (8).jpg
./dataset/images/left/left (25).jpg
./dataset/images/left/left

# 2) load data

In [5]:
Image = []
path_images = './dataset_processed/images'
for mainDir, subDir, fileList in os.walk(path_images):
    for file in fileList:
        currentPath = os.path.join(mainDir, file)
        Image.append(cv2.imread(currentPath)[:, :, 0])
Image = np.array(Image)
dataset_size, H, W = Image.shape

Label = []
path_labels = './dataset_processed/labels'
for file in os.listdir(path_labels):
    Label.append(np.loadtxt(os.path.join(path_labels, file)))
Label = np.array(list(itertools.chain.from_iterable(Label)))
num_classes = int(np.max(Label))+1

# 3) Build Convolution Neural Network

In [6]:
import torch.nn as nn

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1) # 卷积层1，输入通道数为3，输出通道数为16，卷积核大小为3x3，填充为1
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1) # 卷积层2，输入通道数为16，输出通道数为32，卷积核大小为3x3，填充为1
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1) # 卷积层3，输入通道数为32，输出通道数为64，卷积核大小为3x3，填充为1
        self.pool = nn.MaxPool2d(2, 2) # 池化层，池化核大小为2x2，步长为2
        self.fc1 = nn.Linear(64 * 48 // 8 * 48 // 8, 256) # 全连接层1，输入特征数为64 * w // 8 * h // 8，输出特征数为256
        self.fc2 = nn.Linear(256, num_classes) # 全连接层2，输入特征数为256，输出特征数为num_classes
        self.dropout = nn.Dropout(0.2) # 随机失活层，失活概率为0.2

    
    def forward(self, x):
        x=x.unsqueeze(1)
        x = self.pool(torch.relu(self.conv1(x))) # 卷积层1后接激活函数和池化层
        x = self.pool(torch.relu(self.conv2(x))) # 卷积层2后接激活函数和池化层
        x = self.pool(torch.relu(self.conv3(x))) # 卷积层3后接激活函数和池化层
        x = x.view(-1, 64 * 48 // 8 * 48 // 8) # 将张量展平为一维向量
        x = torch.relu(self.fc1(x)) # 全连接层1后接激活函数
        x = self.dropout(x) # 随机失活层
        x = self.fc2(x) # 全连接层2
        return x

In [7]:
import torch.optim as optim

model = CNNModel()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
loss_func = nn.CrossEntropyLoss()

# 4) Train and test

In [8]:
import torch

dataset = utils_data.TensorDataset(torch.Tensor(Image), torch.LongTensor(Label))
split_ratio = 0.8
train_size = int(split_ratio * dataset_size)
test_size = dataset_size - train_size
train_set, test_set = utils_data.random_split(dataset, [train_size, test_size])
train_loader = utils_data.DataLoader(dataset=train_set, batch_size=8, shuffle=True)
test_loader = utils_data.DataLoader(dataset=test_set, batch_size=8, shuffle=True)
print('Data is ready!')

Data is ready!


In [9]:
best_accuracy = 0
for epoch in range(50):
    running_loss = 0.0
    train_acc = 0.0
    for step, (batch_image, batch_label) in enumerate(train_loader):
        model.train()
        # if torch.cuda.is_available():
        #     batch_image, batch_label = batch_image.cuda(), batch_label.cuda()
        batch_output = model(batch_image)
        batch_loss = loss_func(batch_output, batch_label)

        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()

        running_loss += batch_loss.item()

        # train accuracy
        _, train_predicted = torch.max(batch_output.data, 1)
        train_acc += (train_predicted == batch_label).sum().item()

    train_acc /= train_size
    running_loss /= (step+1)

    # ----------test----------
    model.eval()
    test_acc = 0.0
    for test_image, test_label in test_loader:
        test_output = model(test_image)
        _, predicted = torch.max(test_output.data, 1)
        test_acc += (predicted == test_label).sum().item()
    test_acc /= test_size

    print('epoch={:d}\ttrain loss={:.6f}\ttrain accuracy={:.3f}\ttest accuracy={:.3f}'.format(
        epoch, running_loss, train_acc, test_acc))

    if test_acc >= best_accuracy:
        save_path = './trained_cnn/'
        if not os.path.exists(save_path):
             os.makedirs(save_path)
        torch.save(model.state_dict(), './trained_cnn/FCNN_model.pkl')
        best_accuracy = test_acc

epoch=0	train loss=15.518861	train accuracy=0.403	test accuracy=0.812
epoch=1	train loss=0.731783	train accuracy=0.694	test accuracy=0.688
epoch=2	train loss=0.396401	train accuracy=0.887	test accuracy=0.750
epoch=3	train loss=0.275968	train accuracy=0.887	test accuracy=0.688
epoch=4	train loss=0.187599	train accuracy=0.935	test accuracy=0.750
epoch=5	train loss=0.066338	train accuracy=0.984	test accuracy=0.812
epoch=6	train loss=0.068913	train accuracy=0.968	test accuracy=0.812
epoch=7	train loss=0.032775	train accuracy=1.000	test accuracy=0.750
epoch=8	train loss=0.032821	train accuracy=1.000	test accuracy=0.812
epoch=9	train loss=0.015659	train accuracy=1.000	test accuracy=0.750
epoch=10	train loss=0.003571	train accuracy=1.000	test accuracy=0.750
epoch=11	train loss=0.007298	train accuracy=1.000	test accuracy=0.750
epoch=12	train loss=0.007399	train accuracy=1.000	test accuracy=0.688
epoch=13	train loss=0.054677	train accuracy=0.984	test accuracy=0.750
epoch=14	train loss=0.060355	